In [1]:
import numpy as np
from collections import defaultdict
import json

In [2]:
# all this +/- stuff is only really used for the 'current year MOV' prediction
iv = np.array([71.3 ,  6.65, 17.4 ])

def get_mov(x):
    offset = len(iv)//3
    tot = -1.977479789
    for i in range(offset):
        tot += (iv[offset+i])*(np.tanh((x-iv[i])/iv[2*offset+i])+1.0)
    return tot

# estimated mov per ovr
o2m = {}
for i in np.arange(101):
    o2m[i] = get_mov(i)

# replacement level
r_lvl = -1.425


age_shift ={19: 6.4,
 20: 6.7,
 21: 5.0,
 22: 4.4,
 23: 2.2,
 24: 2.0,
 25: 0.1,
 26: -0.1,
 27: -1.2,
 28: -1.4,
 29: -2.0,
 30: -2.7,
 31: -3.6,
 32: -3.7,
 33: -3.7,
 34: -4.7,
 35: -5.1}

max_shift = min(age_shift.values())

# replacement level
RL = 45.3

# expected mov, weight for players, weight for salary cap space
weights = {
0 : ( [0.759, 1.021] , 1.26 ),
1 : ( [11.133, 0.709] , 1.89 ),
2 : ( [9.198, 0.478] , 2.77 ),
}

# draft value
draftP = np.array([0.33331122, 0.30847174, 0.58789643])

# team value
team_mov = [-0.17330874,  0.33627302, 99.72968488]

# salary multiplier
sA = 3.148966624999395

In [3]:
team = { 'mov': -4, 
        'p' : [(22,66,22000,2),(22,66,22000,2),(22,66,22000,2),(22,66,22000,5)], #(age,ovr,salary,years_left)
         'd' : [(1, 0, 8.6),(1, 0, -5.35),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
def get_team_value(team,sCap=90000,minS=750):
    YEARS_TO_MODEL = 3
    teamNum = 30
    CON_BUMP = 0.25
    
    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip( (teamNum-1)/(1+np.exp(0.0036 - 0.368*(x))),0,teamNum-1)))
    m2next = lambda year,mov: [1,0.5,0.25,0.08,0.03,0.01,0.01,0.01,0.01,0.01][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+teamNum*(rnd-1)) for rnd,yr,mov in team['d']]
    
    con_value = 0

    for age,ovr,con,yrl in team['p']:
        ovr2  =ovr
        povrs = [ovr2]
        for j in range(max(0,yrl-1)):
            ovr2+=age_shift.get(age+j,max_shift)
            povrs.append(ovr2)
        pmovs = [o2m[int(np.clip(np.round(ovr2),0,100))] for ovr2 in povrs]
        ccont = [(sCap/3)*(pmov-r_lvl)/sA for pmov in pmovs]
        cvals = [c-con for c in ccont]      
        con_value += sum(cvals)
    for yr,p in draft_picks:
        con_value += sCap * draftP[1]*np.exp(-draftP[0]*p**draftP[2])
        
    return con_value
get_team_value(team,90000,750)

377575.38701186946

In [4]:
data = json.load(open('real_2020.json','rt',encoding='utf-8-sig'))
gA = {_['key']:_['value'] for _ in data['gameAttributes']}

In [5]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']
maxS = gA['maxContract']

team_injuries = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        team_players[p['tid']].append((age,ovr,salary,years_left))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])


In [6]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        if ts['playoffs']:
            continue
        current_mov = 0
        gp = ts['gp']+1e-9
        gl = 82-gp+1e-9
        if season == ts['season'] and not ts['playoffs'] and ts['gp']>0:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = sum(sorted([o2m[_[1]]*(max(gl-i,0)/gl) for _,i in zip(team_players[tid],team_injuries[tid])])[-10:])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov    

In [7]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [8]:
teams_vals = []
for i in range(len(data['teams'])):
    #print(team_names[i])
    val = get_team_value({'mov':team_movs[i],'p':team_players[i],'d':team_picks[i]},sCap,minS)
    teams_vals.append((val,team_names[i]))

In [9]:
for v,t in sorted(teams_vals)[::-1]:
    print(round(100*v,2),t)

25933586.85 NOL
22697569.8 IND
19230977.82 BOS
18595309.09 LAC
18521078.75 LAL
17514628.29 DEN
16274592.37 TOR
15502195.92 MIL
15472464.45 DAL
15406989.9 ATL
15396678.33 MEM
15394983.45 HOU
14891248.17 PHI
12693144.48 MIA
10490361.38 PHO
9839133.9 MIN
9821774.69 NYC
9797975.1 BKN
9742503.88 SAC
9620528.23 CHI
9542426.12 OKC
9294977.77 ORL
6085824.16 POR
5983818.18 CHA
4128026.01 SA
3991281.06 DET
1364734.44 UTA
-644643.21 CLE
-852475.42 WAS
-6409827.69 GS


In [10]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1], 'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2], 'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    pn = np.random.randint(len(team_picks[t1]))
    picks = [_ for _ in team_picks[t1]]
    pick = picks[pn]        
    del picks[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':team_players[t1] + [player],'d':picks},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p,'d':team_picks[t2] + [pick]},sCap,minS)
        vc =  lambda x,y: abs(x-y)/max(abs(x),abs(y))
        val = min(abs(t1v-t1vo),abs(t2v-t2vo))
        deals.append((val,team_names[t1],pick,team_names[t2],player))

In [11]:
sorted(deals)

[(0.20108053792500868,
  'ATL',
  (1, 0, -5.372719925920409),
  'BKN',
  (26, 59, 17500, 3)),
 (8.274164544563973,
  'BOS',
  (1, 2, 6.064637683598374),
  'LAL',
  (30, 53, 4900, 1)),
 (10.265084612648934,
  'DET',
  (1, 2, -8.934317056977779),
  'LAC',
  (34, 56, 6200, 1)),
 (10.265084612648934,
  'OKC',
  (1, 0, -2.314506313566077),
  'LAC',
  (34, 56, 6200, 1)),
 (10.265084612648934,
  'PHO',
  (1, 0, -1.9928904382584818),
  'LAC',
  (34, 56, 6200, 1)),
 (12.501818469638238,
  'CHI',
  (2, 0, -1.7005351319764517),
  'LAC',
  (23, 48, 3750, 2)),
 (13.760879364461289,
  'PHO',
  (2, 2, -1.9928904382584818),
  'OKC',
  (26, 48, 1900, 0)),
 (14.41447255879757,
  'HOU',
  (1, 1, 9.030763602443432),
  'MIN',
  (25, 51, 3300, 0)),
 (17.963319088390563,
  'CHA',
  (2, 1, 10.052609922322244),
  'TOR',
  (23, 47, 1500, 1)),
 (23.507123416837203,
  'GS',
  (1, 0, -4.91664629842538),
  'WAS',
  (23, 55, 8350, 2)),
 (25.416819212870905,
  'LAC',
  (2, 6, 10.052609922322244),
  'ORL',
  (28, 57, 

In [12]:
team_names

{0: 'BOS',
 1: 'NYC',
 2: 'GS',
 3: 'SAC',
 4: 'LAL',
 5: 'DET',
 6: 'PHI',
 7: 'ATL',
 8: 'WAS',
 9: 'CHI',
 10: 'OKC',
 11: 'HOU',
 12: 'MIL',
 13: 'PHO',
 14: 'LAC',
 15: 'CLE',
 16: 'POR',
 17: 'UTA',
 18: 'BKN',
 19: 'SA',
 20: 'DEN',
 21: 'IND',
 22: 'DAL',
 23: 'NOL',
 24: 'MIA',
 25: 'ORL',
 26: 'MIN',
 27: 'TOR',
 28: 'MEM',
 29: 'CHA'}

In [13]:
team_test = {'mov':team_movs[7],'d':team_picks[7],'p':team_players[7]}

In [14]:
get_team_value(team_test,sCap,minS)

154069.89904319722

In [15]:
team_test2 = {'mov': -5.372719925920409,
 'd': [(1, 0, -5.372719925920409),
  (2, 0, -5.372719925920409),
  (2, 0, 9.030763602443432),
  (1, 1, -5.372719925920409),
  (2, 1, -4.958921830230271),
  (2, 1, 4.178927470632469),
  (1, 2, -5.372719925920409),
  (2, 2, -5.372719925920409),
  (2, 2, -4.958921830230271),
  (1, 3, -5.372719925920409),
  (2, 3, -5.372719925920409),
  (2, 3, -7.548780876962224),
  (1, 4, -5.372719925920409),
  (2, 4, -5.372719925920409),
  (1, 5, -5.372719925920409),
  (2, 5, -5.372719925920409),
  (2, 5, -0.5734780821022222),
  (1, 6, -5.372719925920409),
  (2, 6, -5.372719925920409),
  (2, 6, -4.91664629842538)],
 'p': [(31, 50, 13350, 1),
  (26, 58, 17500, 2),
  (25, 27, 1700, 0),
  (22, 31, 1550, 1),
  (23, 25, 950, 0),
  (21, 51, 4850, 2),
  (22, 56, 4250, 1),
  (22, 58, 6650, 1),
  (23, 52, 8050, 2),
  (23, 59, 4150, 0),]}
get_team_value(team_test2,sCap,minS)

128185.93034499981

In [16]:
team_test2 = {'mov': -5.372719925920409,
 'd': [
  (2, 0, -5.372719925920409),
  (2, 0, 9.030763602443432),
  (1, 1, -5.372719925920409),
  (2, 1, -4.958921830230271),
  (2, 1, 4.178927470632469),
  (1, 2, -5.372719925920409),
  (2, 2, -5.372719925920409),
  (2, 2, -4.958921830230271),
  (1, 3, -5.372719925920409),
  (2, 3, -5.372719925920409),
  (2, 3, -7.548780876962224),
  (1, 4, -5.372719925920409),
  (2, 4, -5.372719925920409),
  (1, 5, -5.372719925920409),
  (2, 5, -5.372719925920409),
  (2, 5, -0.5734780821022222),
  (1, 6, -5.372719925920409),
  (2, 6, -5.372719925920409),
  (2, 6, -4.91664629842538)],
 'p': [(20,50,7500,3),
    (31, 50, 13350, 1),
  (26, 58, 17500, 2),
  (25, 27, 1700, 0),
  (22, 31, 1550, 1),
  (23, 25, 950, 0),
  (21, 51, 4850, 2),
  (22, 56, 4250, 1),
  (22, 58, 6650, 1),
  (23, 52, 8050, 2),
  (23, 59, 4150, 0),]}
get_team_value(team_test2,sCap,minS)

147072.4997538393

In [17]:
team_test = {'mov':team_movs[2],'d':team_picks[2],'p':team_players[2]}
get_team_value(team_test,sCap,minS)

-64098.276926308885

In [18]:
team_test2 = {
    'mov':team_movs[2],
    'd': [(2, 0, 1.652969631948502),
 (2, 0, 0.05698951843920175),
 (2, 1, 4.407755127986256),
 (1, 1, -4.91664629842538),
 (1, 1, -3.306876503590236),
 (2, 1, -3.306876503590236),
 (1, 2, -4.91664629842538),
 (2, 2, -4.91664629842538),
 (2, 2, 6.825253437957853),
 (1, 3, -4.91664629842538),
 (1, 5, -4.91664629842538),
 (2, 5, -4.91664629842538),
 (1, 6, -4.91664629842538)],
 'p': [(24, 51, 4800, 2),
 (24, 55, 1650, 2),
 (25, 59, 29550, 3),
 (27, 22, 1650, 2),
 (21, 25, 3900, 3),
 (30, 59, 20200, 4),
 (30, 64, 38000, 4),
 (32, 68, 40250, 2),
 (26, 26, 1650, 2),
 (23, 46, 1800, 1),
 (27, 26, 100, 0),
 (20, 33, 1750, 3),
 (23, 38, 200, 0),
 (28, 38, 1850, 2),
 (23, 29, 1650, 2)]}
get_team_value(team_test2,sCap,minS)

-79954.96413422092